In [3]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from PIL import Image
import math
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
pwd

u'/Users/gc'

In [5]:
cd /Users/gc/Dataset_A

/Users/gc/Dataset_A


In [6]:
path = './data/'         # 替换为你的路径
dir = os.listdir(path)                  # dir是目录下的全部文件
fopen = open('train_img_path.txt', 'w') # 替换为你的路径
for d in dir:                        # d是每一个文件的文件名
    string = 'data/' + d + '\n'    #拼接字符串并换行
    fopen.write(string)             # 写入文件中
fopen.close()

In [7]:
def readlabel(filename1, filename2):
    f1 = open(filename1,'r')
    f2 = open(filename2,'r')
    labellines1=f1.readlines()
    labellines2=f2.readlines()
    
    images=[]
    labels=[]
    path=0
    for line in labellines1:
        for lines in labellines2:
            filename=line[0:-4]
            pathname=lines[5:-35]
            label=line[-3]
            if filename == pathname:
                path=lines[0:-1]
                im=Image.open(path)
                im=np.asarray(im)
                images.append(im)
                labels.append(label)
    images=np.array(images)
    return images,labels
            

In [8]:
train_image,train_label=readlabel('train.txt','train_img_path.txt')

In [10]:
train_image.shape

(410,)

In [11]:
def readtxt(filename1, filename2):
    f1 = open(filename1,'r')
    f2 = open(filename2,'r')
    labellines1=f1.readlines()
    labellines2=f2.readlines()
    
    images=[]
    labels=[]
    path=0
    for line in labellines1:
        for lines in labellines2:
            filename=line[0:-3]
            pathname=lines[5:-35]
            label=line[-3]
            if filename == pathname:
                path=lines[0:-1]
                im=Image.open(path)
                im=np.asarray(im)
                images.append(im)
                labels.append(label)
    images=np.array(images)
    return images,labels

In [12]:
val_image,val_label=readtxt('val.txt','train_img_path.txt')

In [29]:
def readtst(filename1, filename2):
    f1 = open(filename1,'r')
    f2 = open(filename2,'r')
    labellines1=f1.readlines()
    labellines2=f2.readlines()
    
    images=[]
    path=0
    for line in labellines1:
        for lines in labellines2:
            filename=line[0:-2]
            pathname=lines[5:-35]
            if filename == pathname:
                path=lines[0:-1]
                im=Image.open(path)
                im=np.asarray(im)
                images.append(im)
    images=np.array(images)
    return images

In [30]:
test_image=readtst('test.txt','train_img_path.txt')

In [31]:
test_image

array([ array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
       array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
       array([[  0,   0,   0, ...,   0,   0,   0],
       [177, 174, 175, ...,   0,   0,   0],
       [166, 176, 168, ...,   0,   0,   0],
       ..., 
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8),
       array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., 70, 67, 68],
       [ 0,  0,  0, ..., 71, 70, 70],
       ..., 
       [ 0,  0,  0, ..., 77, 76, 75],
       [ 0,  0,  0, ..., 76, 77, 74],
       [ 0,  0,  0, ..., 77, 78, 78]

In [17]:
def patch(image):
    height = 80
    width = 80
    
    images=[]
    for pic in image:
        row =int(math.floor(pic.shape[0]/height))
        col =int(math.floor(pic.shape[1]/width))
        patch_based_image = np.zeros(shape=(row,col,height,width))
        
        picture=[]
        for h in range(0,row):
            for w in range(0,col):
                current_grid = pic[h*height:(h+1)*height,w*width:(w+1)*width]
                picture.append(current_grid)
            
        picture=np.array(picture)
        
        images.append(picture)
    images=np.array(images)
    return images     

In [37]:
train_patch=patch(train_image)
val_patch=patch(val_image)

In [38]:
test_patch=patch(test_image)

In [32]:
test_patch.shape[0]

40

In [57]:
x_train=[]
for x in train_patch:
    x = x.astype('float32')
    x /=255
    x_train.append(x)
#    print(x.shape)
x_train=np.array(x_train)
print(x_train)

[ array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.

In [59]:
for x in x_train:
    print(x.shape)

(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80,

In [58]:
x_val=[]
for x in val_patch:
    x = x.astype('float32')
    x /=255
    x_val.append(x)
#    print(x.shape)
x_val=np.array(x_val)
print(x_test)

[ array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.

In [55]:
x_test=[]
for x in test_patch:
    x = x.astype('float32')
    x /=255
    x_test.append(x)
#    print(x.shape)
x_test=np.array(x_test)
print(x_test)

[ array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.

In [56]:
for x in x_test:
    print(x.shape)

(2091, 80, 80)
(2091, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(2091, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
(1312, 80, 80)
